### 3. Serializa el modelo seleccionado (puede ser de los construidos en el punto 2) e implementa una API REST para poder predecir atrasos de nuevos vuelos.

Dato: en la creacion del modelo de XGBoost la variable a predecir "labels_" está vacía por lo que se solucionó esto

Se serealizó el modelo de **XGBoost después del GridCV realizado por Juan**. Utilicé Flask para implementar el modelo y crear una API debido a que Flask es un framework ligero y fácil de usar para el desarrollo de aplicaciones web en Python.

In [35]:
#importing libraries
import os
import numpy as np
import flask
import pickle
from flask import Flask, render_template, request
#creating instance of the class
app=Flask(__name__)
#to tell flask what url shoud trigger the function index()
@app.route('/')
@app.route('/index')
def index():
  return flask.render_template('index.html')
def ValuePredictor(to_predict_list):
  to_predict = np.array(to_predict_list).reshape(1, 37)
  loaded_model = pickle.load(open("XGBoost_GridCV2.pkl","rb"))
  result = loaded_model.predict(to_predict)
  return result[0]
@app.route('/result', methods = ['POST'])
def result():
  if request.method == 'POST':
    to_predict_list = request.form.to_dict()
    to_predict_list = list(to_predict_list.values())
    try:
      to_predict_list = list(map(float, to_predict_list))
      result = ValuePredictor(to_predict_list)
      if int(result)==0:
        prediction='No atraso'
      elif int(result)==1:
        prediction='Probabilidad de Vuelo con Retraso'
      else:
        prediction=f'{int(result)} No-definida'
    except ValueError:
      prediction= print(request.form.to_dict())
    return render_template("result.html", prediction=prediction)
if __name__=="__main__":
  app.run(port=5006)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5006/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 19:25:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 19:25:21] "POST /result HTTP/1.1" 200 -


**En este punto hay muchas cosas que se pueden mejorar. Empezando por el index, donde se le muestra al usuario todas las opciones de vuelo existentes, donde sería mucho mejor dejarlo como un menu desplegable en el cual solo deba escoger:**
1. Aerolinea
2. Tipo de Vuelo
3. Mes  

Y que este 'form' asuma que los seleccionados corresponden a 1 y los que no han sido seleccionados son 0 para el input del modelo.
Dejé por default que todos los campos fueran 0 para evitar que este deba rellenar las 37 variables.